# BAX pore assembly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("./scripts/output/notebooks/custom_style.mplstyle")
import xarray as xr

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

%matplotlib inline

In [ ]:
slice_vec = np.array([12, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0])

## Marginal and sliced distributions

### Load data

#### Tree tensor integrator

##### Partition 0
(0 1 2)(((3 4 6 7)(5 8))(9 10))

In [ ]:
tree = readTree("output/bax_p0_r5-10-20_i_tau5e-2/output_t2900.nc")
P_slice_tensor0, P_sum_tensor0 = tree.calculateObservables(0, slice_vec)

##### Partition 1
((0 1 2)(3 4 5))((6 7 9)(8 10))

In [ ]:
tree = readTree("output/bax_p1_r5-15-25_i_tau5e-2/output_t2900.nc")
P_slice_tensor10, P_sum_tensor10 = tree.calculateObservables(0, slice_vec)
P_slice_tensor11, P_sum_tensor11 = tree.calculateObservables(1, slice_vec)

##### Partition 2
((0 1)(2 3 4))((5 6 7 8)(9 10))

In [ ]:
tree = readTree("output/bax_p2_r5-25-25_i_tau5e-2/output_t2900.nc")
P_slice_tensor2, P_sum_tensor2 = tree.calculateObservables(0, slice_vec)

##### Two partitions
(0 1 2 3 4)(5 6 7 8 9 10)

In [ ]:
tree = readTree("output/bax_pfull_r5_i_tau5e-2/output_t2900.nc")
P_slice_tensor_full, P_sum_tensor_full = tree.calculateObservables(0, slice_vec)

#### Matrix integrator

In [ ]:
with xr.open_dataset("output/bax_general_230730_r5_so_tau1e0_ss100/output_t100.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    idx_2D = np.array([0, 1])
    P_sum_matrix = lr_sol.marginalDistributions()
    P_slice_matrix = lr_sol.slicedDistributions(slice_vec)

#### SSA

In [ ]:
result = np.load("scripts/reference_solutions/bax_ssa_general_ref_1e6.npy")
ssa_sol = SSASol(result)
P_sum_ssa, _, P_slice_ssa, _ = ssa_sol.calculateObservables(slice_vec, idx_2D)

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/bax_ssa_general_ref_1e5.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
P_sum_ssa_1e5, _, P_slice_ssa_1e5, _ = ssa_1e5_sol.calculateObservables(slice_vec, idx_2D)

### Sliced distributions

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5), sharex='col', sharey='row')
axs[0, 0].plot(np.arange(tree.root.child[0].grid.n[0])[11:26], P_slice_tensor10[11:26], '<-', label="PS-TTN integrator ($P_1$)")
axs[0, 0].plot(np.arange(grid.n[0])[11:26], P_slice_matrix[0][11:26], '^', label="DLRA")
axs[0, 0].plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0])[8:23], P_slice_ssa[-1][0][8:23], 'v', label="SSA ($10^7$ runs)")
axs[0, 0].set_ylabel("$P_\\mathrm{{S}}(x_0)$")
axs[0, 0].set_yscale("log")

axs[1, 0].plot(np.arange(tree.root.child[0].grid.n[0])[11:26], P_slice_tensor10[11:26], '<-')
axs[1, 0].plot(np.arange(grid.n[0])[11:26], P_slice_matrix[0][11:26], '^')
axs[1, 0].plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0])[8:23], P_slice_ssa[-1][0][8:23], 'v')
axs[1, 0].plot((np.arange(ssa_1e5_sol.n[0])+ssa_1e5_sol.n_min[0])[8:23], P_slice_ssa_1e5[-1][0][8:23], '>')
axs[1, 0].set_xlabel(xlabel="$x_0$")
axs[1, 0].set_ylabel("$P_\\mathrm{{S}}(x_0)$")
axs[1, 0].set_xticks(np.arange(11, 25+2, 2))
axs[1, 0].set_xlim([10, 26])

axs[0, 1].plot(np.arange(tree.root.child[0].grid.n[1])[8:16], P_slice_tensor11[8:16], '<-')
axs[0, 1].plot(np.arange(grid.n[1])[8:16], P_slice_matrix[1][8:16], '^')
axs[0, 1].plot((np.arange(ssa_sol.n[1])+ssa_sol.n_min[1])[8:16], P_slice_ssa[-1][1][8:16], 'v')
axs[0, 1].set_xlabel(xlabel="$x_1$")
axs[0, 1].set_ylabel("$P_\\mathrm{{S}}(x_1)$")
axs[0, 1].set_yscale("log")

axs[1, 1].plot(np.arange(tree.root.child[0].grid.n[1])[8:16], P_slice_tensor11[8:16], '<-')
axs[1, 1].plot(np.arange(grid.n[1])[8:16], P_slice_matrix[1][8:16], '^')
axs[1, 1].plot((np.arange(ssa_sol.n[1])+ssa_sol.n_min[1])[8:16], P_slice_ssa[-1][1][8:16], 'v')
axs[1, 1].plot((np.arange(ssa_1e5_sol.n[1])+ssa_1e5_sol.n_min[1])[8:16], P_slice_ssa_1e5[-1][1][8:16], '>')
axs[1, 1].set_xlabel(xlabel="$x_1$")
axs[1, 1].set_ylabel("$P_\\mathrm{{S}}(x_1)$")
axs[1, 1].set_xticks(np.arange(8, 16))
axs[1, 1].set_xlim([7, 16])

axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")
fig.legend(*axs[0, 0].get_legend_handles_labels(), ncols=3, loc="upper center")
plt.subplots_adjust(hspace=.0, wspace=.0)

### Marginal distributions

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
ax1.plot(np.arange(tree.root.child[0].grid.n[0]), P_sum_tensor10, '<-', label="PS-TTN integrator ($P_1$)")
ax1.plot(np.arange(grid.n[0]), P_sum_matrix[0], '^', label="DLRA")
ax1.plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0]), P_sum_ssa[-1][0], 'v', label="SSA ($10^7$ runs)")
ax1.set_xlabel("$x_0$")
ax1.set_ylabel("$P_\\mathrm{{M}}(x_0)$")

ax2.plot(np.arange(tree.root.child[0].grid.n[1]), P_sum_tensor11, '<-')
ax2.plot(np.arange(grid.n[1]), P_sum_matrix[1], '^')
ax2.plot((np.arange(ssa_sol.n[1])+ssa_sol.n_min[1]), P_sum_ssa[-1][1], 'v')
ax2.set_xlabel("$x_1$")
ax2.yaxis.tick_right()
ax2.yaxis.set_ticks_position("both")
ax2.yaxis.set_label_position("right")
ax2.set_ylabel("$P_\\mathrm{{M}}(x_1)$")
fig.legend(*ax1.get_legend_handles_labels(), ncols=3, loc="upper center");

## Comparison with deterministic solution

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-2")
wall_time = time_series.getWallTime()
concentrations = time_series.calculateMoments()
t = time_series.time

In [ ]:
from pysb.integrate import odesolve
from scripts.models.bax_pysb import model
concentrations_ode = odesolve(model, t)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))
deviation = np.sqrt(concentrations[1]-concentrations[0]**2)
observables = ["oBax1", "oBax2"]
observables_alt = ["$S_0$", "$S_1$", "$S_2$"]
for i, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    color = next(ax1._get_lines.prop_cycler)['color']
    ax1.plot(t, concentrations[0][:, i], '-', label=o_alt, color=color, alpha=0.7)
    ax1.fill_between(t, concentrations[0][:, i]-deviation[:, i], concentrations[0][:, i]+deviation[:, i], color=color, alpha=.2)
    ax1.plot(t, concentrations_ode[o], '--', color=color, alpha=1.0)
ax1.set_ylabel("$\langle x_i(t) \\rangle$")
ax1.set_ylim([0.0, 45.0])

observables = ["oBax3", "oBax4", "oBax5"]
observables_alt = ["$S_2$", "$S_3$", "$S_4$"]
for idx_o, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    color = next(ax1._get_lines.prop_cycler)['color']
    i = idx_o + 2
    ax2.plot(t, concentrations[0][:, i], '-', label=o_alt, color=color, alpha=0.7)
    ax2.fill_between(t, concentrations[0][:, i]-deviation[:, i], concentrations[0][:, i]+deviation[:, i], color=color, alpha=.2)
    ax2.plot(t, concentrations_ode[o], '--', color=color, alpha=1.0)
ax2.set_ylim([0.0, 4])
ax2.yaxis.tick_right()
ax2.yaxis.set_ticks_position("both")
ax2.yaxis.set_label_position("right")
plt.setp((ax1, ax2), xlabel="$t$", xlim=[0.0, 145.0], xticks=[0.0, 50.0, 100.0, 145.0]);

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(ll, []) for ll in zip(*lines_labels)]
fig.legend(lines, labels, ncols=5, loc="upper center");

In [ ]:
plt.figure(5)
for i, o in enumerate(model.observables):
    plt.plot(t, np.abs((concentrations[0][:, i]-concentrations_ode[o.name])), '-')
plt.xlabel("$t$")
plt.ylabel("$|c_i^\mathrm{{CME}}(t) - c_i^\mathrm{{ODE}}(t)|$");

## Error depending on rank and partition

### Load data

#### Partition 0

In [ ]:
DLR_marginal_err00, SSA_marginal_err00 = calculateMarginalDistributionError("output/bax_p0_r5-5-5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err10, SSA_marginal_err10 = calculateMarginalDistributionError("output/bax_p0_r5-10-10_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### Partition 1

In [ ]:
DLR_marginal_err01, SSA_marginal_err01 = calculateMarginalDistributionError("output/bax_p1_r5-5-5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err11, SSA_marginal_err11 = calculateMarginalDistributionError("output/bax_p1_r5-10-10_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err21, SSA_marginal_err21 = calculateMarginalDistributionError("output/bax_p1_r5-15-25_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err31, SSA_marginal_err31 = calculateMarginalDistributionError("output/bax_p1_r5-20-20_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### Partition 2

In [ ]:
DLR_marginal_err02, SSA_marginal_err02 = calculateMarginalDistributionError("output/bax_p2_r5-5-5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err12, SSA_marginal_err12 = calculateMarginalDistributionError("output/bax_p2_r5-10-10_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err22, SSA_marginal_err22 = calculateMarginalDistributionError("output/bax_p2_r5-15-15_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err32, SSA_marginal_err32 = calculateMarginalDistributionError("output/bax_p2_r5-20-20_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### Two partitions

In [ ]:
DLR_marginal_err_full, SSA_marginal_err_full = calculateMarginalDistributionError("output/bax_pfull_r5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### SSA with $10^6$ samples

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/bax_ssa_general_ref_1e5.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
P_sum_ssa_1e5, _, _, _ = ssa_1e5_sol.calculateObservables(slice_vec, idx_2D)

ssa_1e6 = np.load("scripts/reference_solutions/bax_ssa_general_ref_1e6.npy")
ssa_1e6_sol = SSASol(ssa_1e6)
P_sum_ssa_1e6, _, _, _ = ssa_1e6_sol.calculateObservables(slice_vec, idx_2D)

DLR_marginal_err_ssa_1e5 = np.zeros(tree.grid.d())
SSA_marginal_err_ssa_1e5 = np.zeros(tree.grid.d())
DLR_marginal_err_ssa_1e6 = np.zeros(tree.grid.d())
SSA_marginal_err_ssa_1e6 = np.zeros(tree.grid.d())

for i in range(tree.grid.d()):
    DLR_marginal_err_ssa_1e5[i] = np.linalg.norm(P_sum_ssa_1e5[-1][i][:tree.grid.n[i]] - P_sum_matrix[i][ssa_1e5_sol.n_min[i] : ssa_1e5_sol.n_min[i]+ssa_1e5_sol.n[i]], np.inf)
    DLR_marginal_err_ssa_1e6[i] = np.linalg.norm(P_sum_ssa_1e6[-1][i][:tree.grid.n[i]] - P_sum_matrix[i][ssa_1e6_sol.n_min[i] : ssa_1e6_sol.n_min[i]+ssa_1e6_sol.n[i]], np.inf)

### Plot

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

cols = ["partition $P_0$", "partition $P_1$", "partition $P_2$"]
rows = ["$r = (5, 5, 5)$", "$r = (5, 10, 10)$", "$r = (5, 15, 15)$", "$r = (5, 20, 20)$"]
labels = ["$\max_{{x_i}}|P_M(x_i)^\mathrm{{TTN}}-P_M(x_i)^\mathrm{{SSA}}|$", 
          "$\max_{{x_i}}|P_M(x_i)^\mathrm{{TTN}}-P_M(x_i)^\mathrm{{DLRA}}|$"]

fig, axs = plt.subplots(4, 3, figsize=(10, 8), sharex='col', sharey='row')

line0, = axs[0, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err00, '--')
line1, = axs[0, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err00, '.-')
axs[1, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err10, '--')
axs[1, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err10, '.-')
# axs[2, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err20)
# axs[2, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err20)
# axs[3, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err30)
# axs[3, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err30)

axs[0, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err01, '--')
axs[0, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err01, '.-')
axs[1, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err11, '--')
axs[1, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err11, '.-')
axs[2, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err21, '--')
axs[2, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err21, '.-')
axs[3, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err31, '--')
axs[3, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err31, '.-')

axs[0, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err02, '--')
axs[0, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err02, '.-')
axs[1, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err12, '--')
axs[1, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err12, '.-')
axs[2, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err22, '--')
axs[2, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err22, '.-')
axs[3, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err32, '--')
axs[3, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err32, '.-')

# axins = inset_axes(axs[-1, 1], width='65%', height='65%', loc='upper center')
# axins.plot(np.arange(tree.grid.d()), SSA_marginal_err_full, '--')
# axins.plot(np.arange(tree.grid.d()), DLR_marginal_err_full, '.-')
# axins.set_ylim([-0.02*0.6, 0.42*0.6])
# axins.annotate("TTN with 2 partitions\n($r=5$)", xy=(0.5, 1), xytext=(0, -30),
#                 xycoords='axes fraction', textcoords='offset points',
#                 size=11, ha='center', va='baseline')

plt.setp(axs, ylim=[-0.02, 0.42])
# plt.setp(axs[:, 0], ylim=[-0.02, 0.22])
# plt.setp(axs[:, 1], ylim=[-0.02, 0.47])
# plt.setp(axs[:, 2], ylim=[-0.02, 0.34])

pad = 5 # in points

for ax, col in zip(axs[0], cols):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad+5),
                xycoords='axes fraction', textcoords='offset points',
                size=14, ha='center', va='baseline')

for ax, row in zip(axs[:,0], rows):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad-pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size=14, ha='right', va='center')

plt.setp(axs[-1, :], xlabel="species $S_i$", xticks=[0, 2, 4, 6, 8, 10])
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
fig.legend([line0, line1], labels, ncols=2, loc="upper center")
plt.subplots_adjust(hspace=.0, wspace=.0)

#### SSA

In [ ]:
plt.plot(np.arange(tree.grid.d()), DLR_marginal_err_ssa_1e5, '<-')
plt.plot(np.arange(tree.grid.d()), DLR_marginal_err_ssa_1e6, '<-')
plt.ylim([-0.02, 0.42]);

## Mass error

### Load data

#### Partition 0

In [ ]:
time_series = TimeSeries("output/bax_p0_r5-5-5_i_tau5e-2")
mass_err00 = time_series.getMassErr()
time = time_series.time

time_series = TimeSeries("output/bax_p0_r5-10-10_i_tau5e-2")
mass_err10 = time_series.getMassErr()

# time_series = TimeSeries("output/bax_p0_r5-15-15_i_tau5e-2")
# mass_err20 = time_series.getMassErr()

# time_series = TimeSeries("output/bax_p0_r5-20-20_i_tau5e-2")
# mass_err30 = time_series.getMassErr()

#### Partition 1

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-5-5_i_tau5e-2")
mass_err01 = time_series.getMassErr()

time_series = TimeSeries("output/bax_p1_r5-10-10_i_tau5e-2")
mass_err11 = time_series.getMassErr()

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-2")
mass_err21 = time_series.getMassErr()

time_series = TimeSeries("output/bax_p1_r5-20-20_i_tau5e-2")
mass_err31 = time_series.getMassErr()

#### Partition 2

In [ ]:
time_series = TimeSeries("output/bax_p2_r5-5-5_i_tau5e-2")
mass_err02 = time_series.getMassErr()

time_series = TimeSeries("output/bax_p2_r5-10-10_i_tau5e-2")
mass_err12 = time_series.getMassErr()

time_series = TimeSeries("output/bax_p2_r5-15-15_i_tau5e-2")
mass_err22 = time_series.getMassErr()

time_series = TimeSeries("output/bax_p2_r5-20-20_i_tau5e-2")
mass_err32 = time_series.getMassErr()

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5))

axs[0, 0].plot(time, mass_err00, label="$P_0$")
axs[0, 0].plot(time, mass_err01, label="$P_1$")
axs[0, 0].plot(time, mass_err02, label="$P_2$")
axs[0, 0].set_title("$r = (5, 5, 5)$")

axs[0, 1].plot(time, mass_err10)
axs[0, 1].plot(time, mass_err11)
axs[0, 1].plot(time, mass_err12)
axs[0, 1].set_title("$r = (5, 10, 10)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

# axs[1, 0].plot(time, mass_err20)
axs[1, 0].plot(time, mass_err21)
axs[1, 0].plot(time, mass_err22)
axs[1, 0].set_title("$r = (5, 15, 15)$")

# axs[1, 1].plot(time, mass_err30)
axs[1, 1].plot(time, mass_err31)
axs[1, 1].plot(time, mass_err32)
axs[1, 1].set_title("$r = (5, 20, 20)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs, xlabel="time $t$", ylabel="$\Delta m$", xlim=[0.0, 145.0], xticks=[0.0, 50.0, 100.0, 145.0])
plt.subplots_adjust(hspace=0.5)
fig.legend(axs[0, 0].get_legend_handles_labels(), labels, ncols=3, loc="upper center");

## Wall time

### Load data

#### Partition 0

In [ ]:
time_series = TimeSeries("output/bax_p0_r5-5-5_i_tau5e-2")
walltime_00 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p0_r5-10-10_i_tau5e-2")
walltime_10 = time_series.getWallTime()

# time_series = TimeSeries("output/bax_p0_r5-15-15_i_tau5e-2")
# walltime_20 = time_series.getWallTime()

# time_series = TimeSeries("output/bax_p0_r5-20-20_i_tau5e-2")
# walltime_30 = time_series.getWallTime()

#### Partition 1

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-5-5_i_tau5e-2")
walltime_01 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p1_r5-10-10_i_tau5e-2")
walltime_11 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-2")
walltime_21 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p1_r5-20-20_i_tau5e-2")
walltime_31 = time_series.getWallTime()

#### Partition 2

In [ ]:
time_series = TimeSeries("output/bax_p2_r5-5-5_i_tau5e-2")
walltime_02 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p2_r5-10-10_i_tau5e-2")
walltime_12 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p2_r5-15-15_i_tau5e-2")
walltime_22 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p2_r5-20-20_i_tau5e-2")
walltime_32 = time_series.getWallTime()

In [ ]:
#### SSA
walltime_ssa_1e6 = 358
walltime_ssa_1e7 = 2898